for smaller test data just for caso_clinico 1,10,11,12,13,14,15,16,17,18,19,2

In [32]:
import spacy
from find_entity import find_start_end
nlp = spacy.load("es_core_news_md")

def get_entities(text_path, annotation_path):
    with open(text_path, "r", encoding='utf-8') as file:
        text = file.read()
    with open(annotation_path, "r", encoding='utf-8') as file:
        annotations = file.read().splitlines()
    entities = []
    print(annotation_path)
    for annotation in annotations:
        # skip if annotation[0] != 'T'
        print(annotation[0])
        if annotation[0] != "T":
            continue
        split = annotation.split()
        if len(split) > 4:
            _, label, _, _, *entity = split
            joined_entity = ' '.join(entity)
            # now we need to search the text for the entity (only because we worked with GPT, if not then can just say _, label, start, end, *entity)
            start, end = find_start_end(text.lower(), joined_entity.lower())
            entities.append((int(start), int(end), label, joined_entity))
    # close the file
    file.close()
    return (text, {"entities": entities})

# test on a single file
print(get_entities("./bg-train/caso_clinico_radiologia1.txt","./bg-train/caso_clinico_radiologia1.ann")[1])

./bg-train/caso_clinico_radiologia1.ann
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
{'entities': [(0, 10, 'SYMPTOM', 'Anisocoria'), (14, 30, 'BACKGROUND', 'varón de 53 años'), (73, 85, 'PROCEDURE', 'trombectomía'), (90, 98, 'CONDITION', 'oclusión'), (105, 141, 'ANATOMICAL', 'arteria cerebral posterior izquierda'), (156, 162, 'FINDING', 'Trombo'), (105, 141, 'ANATOMICAL', 'arteria cerebral posterior izquierda'), (220, 245, 'FINDING', 'caída del flujo sanguíneo'), (294, 323, 'FINDING', 'caída en el tiempo de drenaje'), (389, 391, 'TEST', 'TC'), (418, 437, 'FINDING', 'aumento de densidad'), (444, 471, 'ANATOMICAL', 'hemiprotuberancia izquierda'), (473, 496, 'ANATOMICAL', 'hemisferios cerebelosos'), (498, 518, 'ANATOMICAL', 'cisternas de la base'), (521, 534, 'ANATOMICAL', 'IV ventrículo'), (550, 576, 'CONDITION', 'transformación hemorrágica'), (579, 605, 'CONDITION', 'extravasación de contraste'), (389, 391, 'TEST', 'TC'), (579, 605, 'FINDING', 'extravasación de contraste'), (677, 689, 'CONDI

In [35]:
# get entities from all .ann and .txt files in a folder
import os
from spacy.tokens import DocBin

directory_unsorted = os.listdir('./bg-train')
dir = []
for file in directory_unsorted:
    if file.endswith('.txt'):
        dir.append(file)
textfiles = sorted(dir)[0:12] # adjust as needed
print(textfiles)

def get_data(textfiles, folder):
    data = []
    for file in textfiles:
        if file.endswith(".txt"):
            text_path = os.path.join(folder, file)
            annotation_path = os.path.join(folder, file.replace(".txt", ".ann"))
            data.append(get_entities(text_path, annotation_path))
    return data

training_data = get_data(textfiles, './bg-train')
db = DocBin() # create a DocBin object which will store the training examples
for text, annotations in training_data:
    doc = nlp(text)
    # debug notes: char_span only works if it is in a token and the annotations must be wrong
    ents = []
    starts_ends = [] # to check for conflicts
    for obj in annotations["entities"]:
        print(obj)
        start = obj[0]
        end = obj[1]
        label = obj[2]
        span = doc.char_span(start, end, label=label)
        if span is None:
            print("Skipping entity", span)
        else:
            print(f"Adding entity: {span.text} [{start}-{end}, {label}]")
            # check for conflicts
            conflicts = False
            for s,e in starts_ends:
                if ((start >= s and end <= e) or (start <= s and end >= e)):
                    conflicts = True
            if conflicts:
                print(f"Conflict detected with span: {span.text} [{start}-{end}, {label}]")
            else:
                ents.append(span)
                starts_ends.append((start, end))
                print(starts_ends)
    doc.ents = ents
    db.add(doc)
db.to_disk("./testing.spacy") # save the DocBin object

['caso_clinico_radiologia1.txt', 'caso_clinico_radiologia10.txt', 'caso_clinico_radiologia11.txt', 'caso_clinico_radiologia12.txt', 'caso_clinico_radiologia13.txt', 'caso_clinico_radiologia14.txt', 'caso_clinico_radiologia15.txt', 'caso_clinico_radiologia16.txt', 'caso_clinico_radiologia17.txt', 'caso_clinico_radiologia18.txt', 'caso_clinico_radiologia19.txt', 'caso_clinico_radiologia2.txt']
./bg-train/caso_clinico_radiologia1.ann
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
./bg-train/caso_clinico_radiologia10.ann
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
./bg-train/caso_clinico_radiologia11.ann
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
./bg-train/caso_clinico_radiologia12.ann
T
T
T
T
T
T
T
T
T
T
T
T
./bg-train/caso_clinico_radiologia13.ann
T
T
T
T
T
T
T
T
T
T
T
T
T
T
./bg-train/caso_clinico_radiologia14.ann
T
T
T
T
T
T
T
T
./bg-train/caso_clinico_radiologia15.ann
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
./bg-train/caso_clinico_radiologia16.ann
T
T
T
T
T
T
T
T
T
T
T
T
T
T
T
./bg-train/caso_clin

To visualise the DocBin object, we can use the displacy module from spacy.

In [38]:
from spacy import displacy

# Convert the DocBin object to a list of Doc objects
docs = list(db.get_docs(nlp.vocab))

# Iterate over the Doc objects and visualize them using displacy
for doc in docs:
    # print the title of the file above the displacy render
    displacy.render(doc, style="ent", jupyter=True)

In [6]:
# export to a csv file for viewing
import csv
with open("train_data.csv", "w") as file:
    writer = csv.writer(file)
    writer.writerow(["text", "entities"])
    writer.writerows(training_data)
# open as a preview of the file to demonstrate the training data

NER pipeline component

In [7]:
print(nlp.pipe_names) # 'ner' is already at the end of our pipeline for the model
ner = nlp.get_pipe("ner")

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


Add the entity labels to the model

In [8]:
for _, annotations in training_data:
    for ent in annotations.get("entities"):
        print(ent)
        ner.add_label(ent[2])

(47, 67, 'SYMPTOM', 'distensión abdominal')
(70, 95, 'SYMPTOM', 'estreñimiento de 72 horas')
(158, 187, 'TEST', 'radiografía de abdomen simple')
(203, 238, 'FINDING', 'marcada distensión de asas de colon')
(243, 266, 'FINDING', 'pérdida de las haustras')
(225, 229, 'FINDING', 'asas')
(77, 92, 'SYMPTOM', 'dolor abdominal')
(115, 133, 'SYMPTOM', 'vómito alimentario')
(191, 202, 'ANATOMICAL', 'mesogastrio')
(208, 229, 'SYMPTOM', 'irritación peritoneal')
(279, 291, 'TEST', 'LDH 294 UI/L')
(315, 335, 'SYMPTOM', 'mal control de dolor')
(45, 73, 'CONDITION', 'litiasis coraliforme derecha')
(86, 144, 'PROCEDURE', 'ureterorrenoscopia flexible con nefrolitectomía percutánea')
(147, 173, 'PROCEDURE', 'se coloca catéter ureteral')
(214, 242, 'SYMPTOM', 'dolor en fosa iliaca derecha')
(256, 267, 'CONDITION', 'apendicitis')
(0, 16, 'CONDITION', 'Varón de 60 años')
(61, 89, 'SYMPTOM', 'dolor en fosa iliaca derecha')
(91, 98, 'SYMPTOM', 'náuseas')
(101, 108, 'SYMPTOM', 'vómitos')
(114, 120, 'SYMPTOM',

In [9]:
# we now have the new labels in the NER pipeline
labels = ner.labels
print(labels)

('AGE', 'ANATOMICAL', 'CONDITION', 'FINDING', 'LOC', 'MISC', 'ORG', 'PER', 'PROCEDURE', 'SYMPTOM', 'TEST', 'TIME')


Train the NER model - have to disable the rest of the pipeline because we are just training the NER model for now

In [12]:
from spacy.util import minibatch, compounding
from spacy.training.example import Example
import random
# ??
from pathlib import Path
from tqdm import tqdm
from __future__ import unicode_literals, print_function

# Disable other pipeline components during training if not NER because this is the only part to test
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

db = DocBin().from_disk("./bg-train.spacy")

# we have already set the new labels in the NER pipeline
with nlp.disable_pipes(*other_pipes): 
    for doc in db.get_docs(nlp.vocab):
        annotations = doc.user_data["annotations"]
        example = Example.from_dict(doc, annotations)
        losses = {}
        for ent in annotations.get("entities", []):
            losses[ent[2]] = 1.0 # set the loss to 1.0 for each annotated entity
        nlp.update([example], drop=0.5, losses=losses)

    # optimizer = nlp.begin_training() 
    # for itn in range(1): 
    #     random.shuffle(training_data)
    #     losses = {}
    #     for text, annotations in tqdm(training_data): # tqdm is a progress bar, idea from source referenced below
    #         nlp.update([text], [annotations], drop=0.5, sgd=optimizer, losses=losses)
    #     print(f"Losses at iteration {itn}: {losses}")

KeyError: 'annotations'

references
https://towardsdatascience.com/train-ner-with-custom-training-data-using-spacy-525ce748fab7
